- соединяем таблицы в postgre

In [14]:
import psycopg2
import os
import pandas as pd
import pickle
import numpy as np

from lxml import etree
from psycopg2 import sql
from pymongo import MongoClient

- чтуобы установить внешний ключ в postgre по inn, в дочерних таблицах не должно быть inn, которых нет в родительской (reestr_msp)
- это долгий запрос в Postgre, поэтому пробую найти нужные inn в python и потом удалить их

In [39]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)

In [35]:
connect.close()

Wall time: 3.49 s

In [26]:
%%time
with connect:
    with connect.cursor() as cur:
        cur.execute('SELECT inn FROM reestr_msp')
        reestr_inn = cur.fetchall()
        cur.execute('SELECT inn FROM debt')
        debt_inn = cur.fetchall()
        cur.execute('SELECT inn FROM tax_offense')
        tax_offense_inn = cur.fetchall()
        # здесь inn не уникален!
        cur.execute('SELECT DISTINCT inn FROM support_msp')
        support_msp_inn = cur.fetchall()

Wall time: 3.49 s


In [27]:
reestr_inn = np.array(reestr_inn)[:, 0]
debt_inn = np.array(debt_inn)[:, 0]
tax_offense_inn = np.array(tax_offense_inn)[:, 0]
support_msp_inn = np.array(support_msp_inn)[:, 0]

In [28]:
len(reestr_inn), len(debt_inn), len(tax_offense_inn), len(support_msp_inn)

(2332299, 1080232, 129827, 381279)

In [48]:
# находим разницу # это те, кто не является малым и средним, или кого уже удалили из реестра...
debt_diff = tuple(set(debt_inn) - set(reestr_inn))
tax_offense_diff = tuple(set(tax_offense_inn) - set(reestr_inn))
support_diff = tuple(set(support_msp_inn) - set(reestr_inn))

In [49]:
len(debt_diff), len(tax_offense_diff), len(support_diff)

(263663, 67104, 24989)

- удаление

Wall time: 12.5 s

In [58]:
%%time
with connect:
    with connect.cursor() as cur:
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('debt'), 
                                  sql.Literal(debt_diff)))
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('tax_offense'), 
                                  sql.Literal(tax_offense_diff)))
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('support_msp'), 
                                  sql.Literal(support_diff)))

Wall time: 12.5 s


- теперь можно устанавливать ограничение

In [59]:
with connect:
    with connect.cursor() as cur:
        cur.execute('ALTER TABLE debt ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')
        cur.execute('ALTER TABLE tax_offense ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')
        cur.execute('ALTER TABLE support_msp ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')    

- это не обязательный шаг, но может пригодится, если собирать данные целиком в одном месте

- про дочерние или родительские организации:
    - нет информации, является ли организация дочерней или родительской (на поверхности не видела такого)
    - т.к. малые и средние - скорее будут дочерними, чем родительскими
    - но есть информация по контрактам и договорам (только по тем, по которым законодательно д.б. отражена информация) и достаточно подробная - т.е. дочки скорее всего будут иметь заключённые договора
    - но я туда не смотрю, чтобы совсем не закопаться.
    
- про связь со стоимостью и объемами проданного топлива численности населения по регионам: есть в данных переписи, забрать проще, чем остальное - тк названия унифицированы.
    